<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/320%EC%A4%84%EC%98%80%EB%8B%A4%20%E3%85%8E%E3%85%8E%E3%85%8Eswimswap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SimSwap
Reference: https://github.com/neuralchen/SimSwap

## Prepare code

In [ ]:
%cd /content
!git clone https://github.com/woctezuma/SimSwap.git
%cd /content/SimSwap/
!git checkout upgrade-insightface

In [ ]:
# PyTorch + TorchVision + NumPy 안정 조합 설치
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install numpy==1.24.4


In [ ]:
!pip install insightface==0.7.3 onnxruntime moviepy imageio==2.34.0 > /dev/null

## Prepare models

In [ ]:
%cd /content/SimSwap

In [ ]:
!wget https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip
!wget -P ./arcface_model https://github.com/woctezuma/SimSwap-colab/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth
!wget https://github.com/neuralchen/SimSwap/releases/download/512_beta/512.zip

In [ ]:
!unzip ./checkpoints.zip  -d ./checkpoints

!unzip 512.zip -d ./checkpoints

!unzip antelope.zip -d ./insightface_func/models/

## Prepare data

### Download

In [ ]:
input_fname = '/content/source.png'
output_fname = '/content/target.png'

### Convert to JPG

Images should not be too large, hence the (arbitrary) limitation of 1024 length.

In [ ]:
def get_new_size(img_size,
                 max_allowed_length = 1024):

  if any(max_allowed_length < sz for sz in img_size):
    long_length = max(img_size)
    ratio = max_allowed_length / long_length
  else:
    ratio = 1.0

  new_img_size = [
                  int(ratio*sz)
                  for sz in img_size
                  ]

  return tuple(new_img_size)

In [ ]:
from PIL import Image

allow_resize = False

for fname in [input_fname, output_fname]:
  jpg_fname = fname.replace('.png', '.jpg')

  try:
    img = Image.open(fname)
  except FileNotFoundError:
    continue

  new_size = get_new_size(img.size, max_allowed_length = 1024)
  if allow_resize:
    print(f'Resizing from {img.size} to {new_size}')
    img.resize(new_size)

  print(f'Saving to {jpg_fname}')
  img.convert('RGB').save(jpg_fname)

jpg_input = input_fname.replace('.png', '.jpg')
jpg_output = output_fname.replace('.png', '.jpg')

In [ ]:
from PIL import Image
import cv2
import numpy as np
import insightface

# SCRFD + ArcFace 모델 로드 (더 강력함)
face_detector = insightface.app.FaceAnalysis(name='buffalo_l')  # ← 기존 'antelope'보다 좋음
face_detector.prepare(ctx_id=0, det_size=(320, 320))

img_path = '/content/source.png'
img = cv2.imread(img_path)

faces = face_detector.get(img)

if len(faces) > 0:
    face = faces[0]
    x1, y1, x2, y2 = face.bbox.astype(int)
    face_crop = img[y1:y2, x1:x2]
    cropped_path = '/content/cpc_ackboo_crop.jpg'
    cv2.imwrite(cropped_path, face_crop)
    print(f'✅ 얼굴 감지 성공! → {cropped_path}')
else:
    print('❌ 얼굴을 찾지 못했습니다.')


In [ ]:
import cv2
import insightface

face_detector = insightface.app.FaceAnalysis(name='buffalo_l')
face_detector.prepare(ctx_id=0, det_size=(640, 640))

img = cv2.imread(jpg_output)  # ← 타겟 이미지
faces = face_detector.get(img)

if not faces:
    print("❌ 타겟 이미지에서 얼굴 인식 실패 → SimSwap 실패 원인")
else:
    print(f"✅ 타겟에서 얼굴 {len(faces)}개 인식됨")


In [ ]:
input_fname = '/content/cpc_ackboo_crop.jpg'

## Run

### Single

In [ ]:
cd /content/SimSwap

In [ ]:
%cd /content/SimSwap
%mkdir -p /content/output/single/

!python test_wholeimage_swapsingle.py \
 --no_simswaplogo \
 --use_mask \
 --crop_size 512 \
 --isTrain false \
 --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /content/output/single/ \
 --gpu_ids 0 > /dev/null


### Multi

In [ ]:
%cd /content/SimSwap
%mkdir -p /content/output/multi/

!python test_wholeimage_swapmulti.py \
 --no_simswaplogo \
 --use_mask \
 --crop_size 512 \
 --isTrain false \
 --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /content/output/multi/ \
 --gpu_ids 0 > /dev/null


In [ ]:
%cd /content/SimSwap
!head -70 test_wholeimage_swapsingle.py | grep -A 10 -B 10 "Face_detect_crop\|app.get"